# Import des librairies et chargement des données

In [63]:
import os

# grammaire abstraite de l'arbre syntaxique de Python
import ast

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.api.types import is_string_dtype, is_numeric_dtype

pd.options.mode.chained_assignment = None  # default='warn'

In [140]:
dossier_data = "data/"
nom_fichier = "batiments_data.csv"
batiments_data = pd.read_csv(dossier_data+nom_fichier)

In [141]:
pd.set_option("display.max_columns", None)
batiments_data.head()

,OSEBuildingID,BuildingType,PrimaryPropertyType,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFABuilding,ENERGYSTARScore,SiteEnergyUse,SteamUse,Electricity,NaturalGas,OtherFuelUse,GHGEmissions,BuildingAge,IsSteamUser,IsElectricityUser,IsNaturalGasUser,IsOtherFuelUser,NumberOfUsedEnergies,MostUsedEnergy,GatheredPrimaryPropertyType,LogSiteEnergyUse,LogGHGEmissions
0,3,NonResidential,Hotel,DOWNTOWN,1969,1.0,41.0,961990.0,18.0,73130656.0,19660404.0,49762435.0,3709900.0,0.0,2061.48,46,1.0,1.0,1.0,0.0,3.0,Electricity,Hotel,18.107758,7.631664
1,5,NonResidential,Hotel,DOWNTOWN,1926,1.0,10.0,61320.0,1.0,28229320.0,23458518.0,2769023.0,2001894.0,0.0,1936.34,89,1.0,1.0,1.0,0.0,3.0,Steam,Hotel,17.155872,7.569071
2,8,NonResidential,Hotel,DOWNTOWN,1980,1.0,18.0,107430.0,67.0,14829099.0,0.0,6066245.0,8763105.0,0.0,507.70,35,0.0,1.0,1.0,0.0,2.0,NaturalGas,Hotel,16.512102,6.231858
3,19,NonResidential,Hotel,DOWNTOWN,1922,1.0,11.0,67390.0,14.0,10711451.0,4403788.0,4089407.0,2218425.0,0.0,486.25,93,1.0,1.0,1.0,0.0,3.0,Steam,Hotel,16.186824,6.188777
4,25,NonResidential,Hotel,DOWNTOWN,1916,1.0,10.0,104352.0,83.0,7845112.0,3205497.0,1790665.0,2849024.0,0.0,411.22,99,1.0,1.0,1.0,0.0,3.0,Steam,Hotel,15.875401,6.021557


# Sélection des variables finales

In [142]:
batiments_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2320 entries, 0 to 2319
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   OSEBuildingID                2320 non-null   int64  
 1   BuildingType                 2320 non-null   object 
 2   PrimaryPropertyType          2320 non-null   object 
 3   Neighborhood                 2320 non-null   object 
 4   YearBuilt                    2320 non-null   int64  
 5   NumberofBuildings            2320 non-null   float64
 6   NumberofFloors               2311 non-null   float64
 7   PropertyGFABuilding          2318 non-null   float64
 8   ENERGYSTARScore              1532 non-null   float64
 9   SiteEnergyUse                2320 non-null   float64
 10  SteamUse                     2320 non-null   float64
 11  Electricity                  2320 non-null   float64
 12  NaturalGas                   2320 non-null   float64
 13  OtherFuelUse      

In [143]:
variables_ecartees = ["OSEBuildingID", "PrimaryPropertyType", "ENERGYSTARScore", "YearBuilt", "SteamUse", "Electricity",
                      "NaturalGas", "OtherFuelUse", "SiteEnergyUse", "GHGEmissions"]
batiments_data_modeles = batiments_data.drop(columns=variables_ecartees)

# Préparation des données

## Variables catégorielles

Pour les variables catégorielles, je vais toper chaque modalité.

In [144]:
batiments_data_modeles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2320 entries, 0 to 2319
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   BuildingType                 2320 non-null   object 
 1   Neighborhood                 2320 non-null   object 
 2   NumberofBuildings            2320 non-null   float64
 3   NumberofFloors               2311 non-null   float64
 4   PropertyGFABuilding          2318 non-null   float64
 5   BuildingAge                  2320 non-null   int64  
 6   IsSteamUser                  2320 non-null   float64
 7   IsElectricityUser            2320 non-null   float64
 8   IsNaturalGasUser             2320 non-null   float64
 9   IsOtherFuelUser              2320 non-null   float64
 10  NumberOfUsedEnergies         2320 non-null   float64
 11  MostUsedEnergy               2320 non-null   object 
 12  GatheredPrimaryPropertyType  2320 non-null   object 
 13  LogSiteEnergyUse  

In [145]:
s = (batiments_data_modeles.dtypes == 'object')
variables_categorielles = list(s[s].index)

In [146]:
modalites_topees = pd.get_dummies(batiments_data_modeles[variables_categorielles], drop_first=True)
batiments_data_topees = batiments_data_modeles.drop(columns=variables_categorielles)
batiments_data_topees = pd.concat([batiments_data_topees, modalites_topees], axis=1)
batiments_data_topees.rename(
    columns={"BuildingType_Nonresidential COS": "BuildingType_NonResidential_COS", 
             "BuildingType_Nonresidential WA": "BuildingType_NonResidential_WA", 
             "BuildingType_SPS-District K-12": "BuildingType_SPS_District_K-12", 
             "Neighborhood_GREATER DUWAMISH": "Neighborhood_GREATER_DUWAMISH", 
             "Neighborhood_LAKE UNION": "Neighborhood_LAKE_UNION", 
             "Neighborhood_MAGNOLIA / QUEEN ANNE": "Neighborhood_MAGNOLIA_QUEEN_ANNE", 
             "GatheredPrimaryPropertyType_Medical and Care": "GatheredPrimaryPropertyType_Medical_and_Care", 
             "GatheredPrimaryPropertyType_Mixed Use Property": "GatheredPrimaryPropertyType_Mixed_Use_Property", 
             "GatheredPrimaryPropertyType_Residence Hall": "GatheredPrimaryPropertyType_Residence_Hall", 
             "GatheredPrimaryPropertyType_Self-Storage Facility": "GatheredPrimaryPropertyType_Self_Storage_Facility", 
             "GatheredPrimaryPropertyType_Worship Facility": "GatheredPrimaryPropertyType_Worship_Facility"},
    inplace=True)

In [147]:
batiments_data_topees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2320 entries, 0 to 2319
Data columns (total 41 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   NumberofBuildings                                  2320 non-null   float64
 1   NumberofFloors                                     2311 non-null   float64
 2   PropertyGFABuilding                                2318 non-null   float64
 3   BuildingAge                                        2320 non-null   int64  
 4   IsSteamUser                                        2320 non-null   float64
 5   IsElectricityUser                                  2320 non-null   float64
 6   IsNaturalGasUser                                   2320 non-null   float64
 7   IsOtherFuelUser                                    2320 non-null   float64
 8   NumberOfUsedEnergies                               2320 non-null   float64
 9   LogSiteE

In [148]:
batiments_data_topees.head()

,NumberofBuildings,NumberofFloors,PropertyGFABuilding,BuildingAge,IsSteamUser,IsElectricityUser,IsNaturalGasUser,IsOtherFuelUser,NumberOfUsedEnergies,LogSiteEnergyUse,LogGHGEmissions,BuildingType_NonResidential,BuildingType_NonResidential_COS,BuildingType_NonResidential_WA,BuildingType_SPS_District_K-12,Neighborhood_CENTRAL,Neighborhood_DELRIDGE,Neighborhood_DOWNTOWN,Neighborhood_EAST,Neighborhood_GREATER_DUWAMISH,Neighborhood_LAKE_UNION,Neighborhood_MAGNOLIA_QUEEN_ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,MostUsedEnergy_NaturalGas,MostUsedEnergy_Steam,GatheredPrimaryPropertyType_Education,GatheredPrimaryPropertyType_Hotel,GatheredPrimaryPropertyType_Medical_and_Care,GatheredPrimaryPropertyType_Mixed_Use_Property,GatheredPrimaryPropertyType_Office,GatheredPrimaryPropertyType_Other,GatheredPrimaryPropertyType_Residence_Hall,GatheredPrimaryPropertyType_Restaurant,GatheredPrimaryPropertyType_Self_Storage_Facility,GatheredPrimaryPropertyType_Store,GatheredPrimaryPropertyType_Warehouse,GatheredPrimaryPropertyType_Worship_Facility
0,1.0,41.0,961990.0,46,1.0,1.0,1.0,0.0,3.0,18.107758,7.631664,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1.0,10.0,61320.0,89,1.0,1.0,1.0,0.0,3.0,17.155872,7.569071,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2,1.0,18.0,107430.0,35,0.0,1.0,1.0,0.0,2.0,16.512102,6.231858,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,1.0,11.0,67390.0,93,1.0,1.0,1.0,0.0,3.0,16.186824,6.188777,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
4,1.0,10.0,104352.0,99,1.0,1.0,1.0,0.0,3.0,15.875401,6.021557,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0


In [149]:
donnees_manquantes = batiments_data_topees.isna().sum()
donnees_manquantes[donnees_manquantes > 0]

NumberofFloors         9
PropertyGFABuilding    2
dtype: int64

## Séparation du dataset en training et test sets

Cette séparation sera faite deux fois : 
- la première pour la consommation d'énergie totale
- la seconde pour les émissions de GES

In [150]:
X = batiments_data_topees.drop(columns=["LogSiteEnergyUse", "LogGHGEmissions"])
y_conso_energie = batiments_data_topees["LogSiteEnergyUse"]
y_emissions = batiments_data_topees["LogGHGEmissions"]

In [151]:
from sklearn.model_selection import train_test_split

# Séparation pour la consommation d'énergie
X_conso_train, X_conso_test, y_conso_train, y_conso_test = train_test_split(X, y_conso_energie)

# Séparation pour les émissions de GES
X_emissions_train, X_emissions_test, y_emissions_train, y_emissions_test = train_test_split(X, y_emissions)

## Imputation des données manquantes

Pour rappel, il reste des données manquantes sur le nombre d'étages et la surface des bâtiments. La première sera imputée par son mode et la seconde par la médiane.

In [152]:
from sklearn.impute import SimpleImputer

imputer_nb_etage = SimpleImputer(strategy="most_frequent")
imputer_surface_batiments = SimpleImputer(strategy="median")

############################################################################################
#                                   Consommation d'énergie
############################################################################################

# Imputation : on donne à l'imputer plusieurs colonnes car ne fonctionne pas avec une seule
imputed_X_conso_train = pd.DataFrame(imputer_nb_etage.fit_transform(X_conso_train[["NumberofBuildings", "NumberofFloors"]]))
imputed_X_conso_train2 = pd.DataFrame(imputer_surface_batiments.fit_transform(X_conso_train[["PropertyGFABuilding", 
                                                                                             "BuildingAge"]]))
imputed_X_conso_test = pd.DataFrame(imputer_nb_etage.transform(X_conso_test[["NumberofBuildings", "NumberofFloors"]]))
imputed_X_conso_test2 = pd.DataFrame(imputer_surface_batiments.transform(X_conso_test[["PropertyGFABuilding", 
                                                                                       "BuildingAge"]]))


# Remise en forme des datasets imputés, remise des noms des colonnes et des index
imputed_X_conso_train = pd.concat([imputed_X_conso_train, imputed_X_conso_train2], axis=1)
imputed_X_conso_test = pd.concat([imputed_X_conso_test, imputed_X_conso_test2], axis=1)

imputed_X_conso_train.columns = ["NumberofBuildings", "NumberofFloors", "PropertyGFABuilding", "BuildingAge"]
imputed_X_conso_test.columns = ["NumberofBuildings", "NumberofFloors", "PropertyGFABuilding", "BuildingAge"]

imputed_X_conso_train.index = X_conso_train.index
imputed_X_conso_test.index = X_conso_test.index

donnees_restantes_train = X_conso_train.drop(columns=["NumberofBuildings", "NumberofFloors", 
                                                      "PropertyGFABuilding", "BuildingAge"])
donnees_restantes_test = X_conso_test.drop(columns=["NumberofBuildings", "NumberofFloors", 
                                                      "PropertyGFABuilding", "BuildingAge"])

imputed_X_conso_train = pd.concat([imputed_X_conso_train, donnees_restantes_train], axis=1)
imputed_X_conso_test = pd.concat([imputed_X_conso_test, donnees_restantes_test], axis=1)

############################################################################################
#                                      Emissions de GES
############################################################################################

# Imputation : on donne à l'imputer plusieurs colonnes car ne fonctionne pas avec une seule
imputed_X_emissions_train = pd.DataFrame(imputer_nb_etage.fit_transform(X_emissions_train[["NumberofBuildings", 
                                                                                           "NumberofFloors"]]))
imputed_X_emissions_train2 = pd.DataFrame(imputer_surface_batiments.fit_transform(X_emissions_train[["PropertyGFABuilding", 
                                                                                                     "BuildingAge"]]))
imputed_X_emissions_test = pd.DataFrame(imputer_nb_etage.transform(X_emissions_test[["NumberofBuildings", "NumberofFloors"]]))
imputed_X_emissions_test2 = pd.DataFrame(imputer_surface_batiments.transform(X_emissions_test[["PropertyGFABuilding", 
                                                                                               "BuildingAge"]]))


# Remise en forme des datasets imputés, remise des noms des colonnes et des index
imputed_X_emissions_train = pd.concat([imputed_X_emissions_train, imputed_X_emissions_train2], axis=1)
imputed_X_emissions_test = pd.concat([imputed_X_emissions_test, imputed_X_emissions_test2], axis=1)

imputed_X_emissions_train.columns = ["NumberofBuildings", "NumberofFloors", "PropertyGFABuilding", "BuildingAge"]
imputed_X_emissions_test.columns = ["NumberofBuildings", "NumberofFloors", "PropertyGFABuilding", "BuildingAge"]

imputed_X_emissions_train.index = X_emissions_train.index
imputed_X_emissions_test.index = X_emissions_test.index

donnees_restantes_train = X_emissions_train.drop(columns=["NumberofBuildings", "NumberofFloors", 
                                                          "PropertyGFABuilding", "BuildingAge"])
donnees_restantes_test = X_emissions_test.drop(columns=["NumberofBuildings", "NumberofFloors", 
                                                        "PropertyGFABuilding", "BuildingAge"])

imputed_X_emissions_train = pd.concat([imputed_X_emissions_train, donnees_restantes_train], axis=1)
imputed_X_emissions_test = pd.concat([imputed_X_emissions_test, donnees_restantes_test], axis=1)

## Standardisation des variables

In [177]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)

############################################################################################
#                                   Consommation d'énergie
############################################################################################

# Standardisation des données
std_X_conso_train = pd.DataFrame(scaler.fit_transform(imputed_X_conso_train))
std_X_conso_test = pd.DataFrame(scaler.transform(imputed_X_conso_test))

# Remise des colonnes et des index du dataset initial
std_X_conso_train.columns = imputed_X_conso_train.columns
std_X_conso_test.columns = imputed_X_conso_test.columns

std_X_conso_train.index = imputed_X_conso_train.index
std_X_conso_test.index = imputed_X_conso_test.index

############################################################################################
#                                   Emissions de GES
############################################################################################

# Standardisation des données
std_X_emissions_train = pd.DataFrame(scaler.fit_transform(imputed_X_emissions_train))
std_X_emissions_test = pd.DataFrame(scaler.transform(imputed_X_emissions_test))

# Remise des colonnes et des index du dataset initial
std_X_emissions_train.columns = imputed_X_emissions_train.columns
std_X_emissions_test.columns = imputed_X_emissions_test.columns

std_X_emissions_train.index = imputed_X_emissions_train.index
std_X_emissions_test.index = imputed_X_emissions_test.index

Toutes nos variables ont un écart-type quasiment égal à 1 dans les jeux d'entraînement.

# Modélisation